In [5]:
import os
os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [98]:
import json
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import random
import pandas as pd
from sklearn.metrics import classification_report
import boto3

In [6]:
from source.utils import flatten_list, create_path, get_keys

In [100]:
with open('./data/few_rel/val_wiki.json','r') as data:
    js_ls = json.load(data)
with open('./data/few_rel/train_wiki.json','r') as data:
    js_ls_tr = json.load(data)

In [107]:
js_ls['P177'][0]

{'tokens': ['In',
  'June',
  '1987',
  ',',
  'the',
  'Missouri',
  'Highway',
  'and',
  'Transportation',
  'Department',
  'approved',
  'design',
  'location',
  'of',
  'a',
  'new',
  'four',
  '-',
  'lane',
  'Mississippi',
  'River',
  'bridge',
  'to',
  'replace',
  'the',
  'deteriorating',
  'Cape',
  'Girardeau',
  'Bridge',
  '.'],
 'h': ['cape girardeau bridge', 'Q5034838', [[26, 27, 28]]],
 't': ['mississippi river', 'Q1497', [[19, 20]]]}

In [104]:
list_rels_tr = list(js_ls_tr)
list_rels = list(js_ls)

In [51]:
set(list_rels_tr) & set(list_rels)

set()

In [105]:
list_rels

['P177',
 'P364',
 'P2094',
 'P361',
 'P641',
 'P59',
 'P413',
 'P206',
 'P412',
 'P155',
 'P26',
 'P410',
 'P25',
 'P463',
 'P40',
 'P921']

In [47]:
len(list_rels)

16

In [116]:
random.sample(range(10),k=4)

[9, 4, 5, 6]

In [121]:
len(all_sents_ex_current)

44100

In [99]:
%%time


def get_results_hf_preds_semeval(method = 'bracket', model_class = 'distilbert-base-uncased',
     train_dataset = 'few_rel'):
    model_path = f'./trained_models/distilbert-base-uncased-mtb-rnd_fel_rel_bracket_pred'
    df_pred = pd.read_csv(f'{model_path}/test_results_None.txt',sep='\t',index_col=0)
    df_test = pd.read_csv(f'./data/train_samples/sem_eval_test_forpred_bracket.csv')
    cls_rpt = classification_report(df_test.label.values,df_pred.prediction.values, output_dict=True)
    return pd.DataFrame(cls_rpt).round(2)

get_results_hf_preds_semeval(method = 'bracket', train_dataset = 'fel_rel')

CPU times: user 22.7 ms, sys: 4.13 ms, total: 26.9 ms
Wall time: 26.9 ms


,0,1,accuracy,macro avg,weighted avg
precision,0.51,0.52,0.51,0.51,0.51
recall,0.64,0.38,0.51,0.51,0.51
f1-score,0.57,0.44,0.51,0.51,0.51
support,1354.00,1354.00,0.51,2708.00,2708.00


In [52]:

#method = 'ignore'



def get_results_hf_preds(method = 'bracket', model_class = 'distilbert-base-uncased',
     train_dataset = 'few_rel', eval_dataset = 'few_rel'):
    model_path = f'./trained_models/{model_class}-mtb-rnd_{train_dataset}_{method}'
    df_pred = pd.read_csv(f'{model_path}/test_results_None.txt',sep='\t',index_col=0)
    df_test = pd.read_csv(f'./data/train_samples/{eval_dataset}_val_{method}_test.csv')
    cls_rpt = classification_report(df_test.label.values,df_pred.prediction.values, output_dict=True)
    return pd.DataFrame(cls_rpt).round(2)

get_results_hf_preds(method = 'bracket', train_dataset = 'fel_rel', eval_dataset = 'few_rel')

,0,1,accuracy,macro avg,weighted avg
precision,0.84,0.86,0.85,0.85,0.85
recall,0.86,0.83,0.85,0.85,0.85
f1-score,0.85,0.84,0.85,0.85,0.85
support,5600.00,5600.00,0.85,11200.00,11200.00


In [42]:
get_results_hf_preds(method = 'mask_one', train_dataset = 'fel_rel', eval_dataset = 'few_rel')

,0,1,accuracy,macro avg,weighted avg
precision,0.79,0.77,0.78,0.78,0.78
recall,0.76,0.80,0.78,0.78,0.78
f1-score,0.77,0.78,0.78,0.78,0.78
support,5600.00,5600.00,0.78,11200.00,11200.00


In [43]:
get_results(method = 'ignore', train_dataset = 'fel_rel', eval_dataset = 'few_rel')

,0,1,accuracy,macro avg,weighted avg
precision,0.80,0.77,0.78,0.78,0.78
recall,0.76,0.81,0.78,0.78,0.78
f1-score,0.78,0.79,0.78,0.78,0.78
support,5600.00,5600.00,0.78,11200.00,11200.00


In [29]:
df_sent = pd.read_csv('./data/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_CLEAN.TXT',
            sep='\t',header=None, index_col=0)

df_key = pd.read_csv('./data/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_KEY.TXT',
            sep='\t',header=None, index_col=0)

In [34]:
df_key = pd.read_csv('./data/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT',
            sep='\t',header=None, index_col=0)

In [93]:
path_file = './data/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT'



In [96]:
pd.read_csv('./data/train_samples/sem_eval_test_forpred_bracket.csv').shape

(2708, 3)

In [87]:
pd.read_csv('./data/train_samples/few_rel_train_bracket_val.csv').columns

Index(['sentence1', 'sentence2', 'label'], dtype='object')

In [95]:
from create_train_data import read_sem_eval

read_sem_eval(path_file)

,sentence,label,comment
i,,,


In [64]:
import re

re.sub('<[/]?e[1,2]>','',df_semeval.sentence.iloc[0])

'"The system as described above has its greatest application in an arrayed configuration of antenna elements."'

In [66]:
for i in range(10):
    print(df_semeval.sentence.iloc[i])
    print(re.sub('<[/]?e[1,2]>','',df_semeval.sentence.iloc[i]))

"The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>."
"The system as described above has its greatest application in an arrayed configuration of antenna elements."
"The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord."
"The child was carefully wrapped and bound into the cradle by means of a cord."
"The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code."
"The author of a keygen uses a disassembler to look at the raw assembly code."
"A misty <e1>ridge</e1> uprises from the <e2>surge</e2>."
"A misty ridge uprises from the surge."
"The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo."
"The student association is the voice of the undergraduate student population of the State University of New York at Buffalo."
"This is the sprawling <e1>complex</e1

In [55]:
if method == 'ignore':
    []

,sentence,label,comment
i,,,
1,"""The system as described above has its greates...","Component-Whole(e2,e1)",Comment: Not a collection: there is structure ...
2,"""The <e1>child</e1> was carefully wrapped and ...",Other,Comment:
3,"""The <e1>author</e1> of a keygen uses a <e2>di...","Instrument-Agency(e2,e1)",Comment:
4,"""A misty <e1>ridge</e1> uprises from the <e2>s...",Other,Comment:
5,"""The <e1>student</e1> <e2>association</e2> is ...","Member-Collection(e1,e2)",Comment:
...,...,...,...
7996,"""When the <e1>notice</e1> is sent by <e2>fax</...",Other,Comment:
7997,"""The <e1>herbicide</e1> is derived from a natu...","Entity-Origin(e1,e2)",Comment:
7998,"""To test this, we placed a kitchen <e1>match</...","Entity-Destination(e1,e2)",Comment:


,1
0,
8001,The most common <e1>audits</e1> were about <e2...
8002,The <e1>company</e1> fabricates plastic <e2>ch...
8003,The school <e1>master</e1> teaches the lesson ...
8004,The suspect dumped the dead <e1>body</e1> into...
8005,Avian <e1>influenza</e1> is an infectious dise...
...,...
10713,"After seating all the idols, which itself take..."
10714,The minister attributed the slow production of...
10715,The <e1>umbrella</e1> <e2>frame</e2> is provid...


In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

model_path = './trained_models/distilbert-base-uncased-mtb-rnd_fel_rel_bracket/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)


In [6]:
tokenizer(df_semeval[1].iloc[0])

{'input_ids': [101, 1996, 2087, 2691, 30522, 15727, 2015, 30523, 2020, 2055, 30524, 5949, 30525, 1998, 17874, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
round(246915368/10e5,1)

246.9

In [137]:

create_path(save_path)

In [138]:
random_state = 1701



In [172]:
model_name = 'distilbert-base-uncased-mtb-rnd'
exp = 'fel_rel_bracket'
create_path('./trained_models/')
p = dict(
    model_path = f'./models/{model_name}/',
    max_len = 128*2,
    tr_ep = 15,
    output_dir = f'./trained_models/{model_name}_{exp}/',
    file = './data/train_samples/few_rel_train_bracket',
    test_mode = True,
    nr_test = 10
)
create_run_glue_script(p)

./trained_models/ exists already
python run_glue.py        --model_name_or_path ./models/distilbert-base-uncased-mtb-rnd/        --do_train        --do_eval        --do_predict        --max_seq_length 256        --num_train_samples 15        --output_dir ./trained_models/distilbert-base-uncased-mtb-rnd_fel_rel_bracket/        --train_file ./data/train_samples/few_rel_train_bracket_train.csv        --validation_file ./data/train_samples/few_rel_train_bracket_val.csv        --test_file ./data/train_samples/few_rel_train_bracket_test.csv --max_train_samples 10        --max_val_samples 10        --max_test_samples 10


In [165]:
python source/add_entity_tokens.py --model_name distilbert-base-uncased
python get_data.py
python create_train_data.py
python 


RangeIndex(start=40320, stop=44800, step=1)

In [152]:
test_len

4480

In [ ]:
main driver

x driven by ...
x caused by ...

examples of x is

In [10]:
sentence = Sentence(s) for s in df_semeval[1][8001]

'The most common <e1>audits</e1> were about <e2>waste</e2> and recycling.'

In [8]:
from flair.data import Sentence
from flair.models import SequenceTagger
# predict tags for sentences
tagger = SequenceTagger.load('ner')


2021-04-22 12:57:24,904 --------------------------------------------------------------------------------
2021-04-22 12:57:24,905 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-04-22 12:57:24,905  - The most current version of the model is automatically downloaded from there.
2021-04-22 12:57:24,906  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-04-22 12:57:24,907 --------------------------------------------------------------------------------
2021-04-22 12:57:25,178 loading file /Users/md/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [18]:
import spacy
nlp = spacy.load("en_core_web_lg")


In [22]:
def get_list_ents(ls_sent):
    res = []
    for txt in ls_sent:
        doc = nlp(txt)

        ent = []
        for entity in doc.ents:
            ent.append({'text':entity.text,'label':entity.label_,'range':[entity.start_char, entity.end_char]})
        res.append({'text':txt,'entities':ent})
    return res

In [28]:
4*mini_batch_size / 30

4.266666666666667

In [26]:
%%time
res = get_list_ents(list(df_semeval[1].values)[:4*mini_batch_size])
for i,s in enumerate(res):
    print(i,s)

0 {'text': 'The most common <e1>audits</e1> were about <e2>waste</e2> and recycling.', 'entities': []}
1 {'text': 'The <e1>company</e1> fabricates plastic <e2>chairs</e2>.', 'entities': []}
2 {'text': 'The school <e1>master</e1> teaches the lesson with a <e2>stick</e2>.', 'entities': []}
3 {'text': 'The suspect dumped the dead <e1>body</e1> into a local <e2>reservoir</e2>.', 'entities': []}
4 {'text': 'Avian <e1>influenza</e1> is an infectious disease of birds caused by type A strains of the influenza <e2>virus</e2>.', 'entities': []}
5 {'text': 'The <e1>ear</e1> of the African <e2>elephant</e2> is significantly larger--measuring 183 cm by 114 cm in the bush elephant.', 'entities': [{'text': 'African', 'label': 'NORP', 'range': [24, 31]}, {'text': '183 cm', 'label': 'QUANTITY', 'range': [85, 91]}, {'text': '114 cm', 'label': 'QUANTITY', 'range': [95, 101]}]}
6 {'text': 'A child is told a <e1>lie</e1> for several years by their <e2>parents</e2> before he/she realizes that a Santa Claus 

In [16]:
%%time
# iterate through sentences and print predicted labels
mini_batch_size = 32
sentences = [Sentence(s) for s in list(df_semeval[1].values)[:4*mini_batch_size]]
tagger.predict(sentences, mini_batch_size=mini_batch_size)
#print(sentence.to_tagged_string())

CPU times: user 38.4 s, sys: 2.45 s, total: 40.9 s
Wall time: 38.1 s


In [24]:
for i,s in enumerate(sentences):
    print(i,s.to_dict(tag_type='ner'))

0 {'text': 'The most common <e1>audits</e1> were about <e2>waste</e2> and recycling.', 'labels': [], 'entities': []}
1 {'text': 'The <e1>company</e1> fabricates plastic <e2>chairs</e2>.', 'labels': [], 'entities': []}
2 {'text': 'The school <e1>master</e1> teaches the lesson with a <e2>stick</e2>.', 'labels': [], 'entities': []}
3 {'text': 'The suspect dumped the dead <e1>body</e1> into a local <e2>reservoir</e2>.', 'labels': [], 'entities': []}
4 {'text': 'Avian <e1>influenza</e1> is an infectious disease of birds caused by type A strains of the influenza <e2>virus</e2>.', 'labels': [], 'entities': []}
5 {'text': 'The <e1>ear</e1> of the African <e2>elephant</e2> is significantly larger--measuring 183 cm by 114 cm in the bush elephant.', 'labels': [], 'entities': [{'text': 'African', 'start_pos': 24, 'end_pos': 31, 'labels': [MISC (0.9657)]}]}
6 {'text': 'A child is told a <e1>lie</e1> for several years by their <e2>parents</e2> before he/she realizes that a Santa Claus does not exist

In [ ]:
- Enter entity token around the sequence
- Replace with equal length entity token E1, E1, \E1
- Replace with single entity token

In [8]:
print(js_ls['P931'][0])

{'tokens': ['Merpati', 'flight', '106', 'departed', 'Jakarta', '(', 'CGK', ')', 'on', 'a', 'domestic', 'flight', 'to', 'Tanjung', 'Pandan', '(', 'TJQ', ')', '.'], 'h': ['tjq', 'Q1331049', [[16]]], 't': ['tanjung pandan', 'Q3056359', [[13, 14]]]}
